In [ ]:
!pip install wheel
!pip install pandas
!pip install nucleus-api

In [2]:
import nucleus_api

configuration = nucleus_api.Configuration()
# configuration.host = 'localhost:5000'
configuration.host = 'http://localhost:5000/'
configuration.api_key['x-api-key'] = 'VRgaGSWm'

api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
import urllib, urllib.request
import sys
import os
import re
import arxiv

def search():
    # Define the base URL for the arXiv API
    base_url = "http://export.arxiv.org/api/query?"
    # we can limit the number of results like this
    nrPapers = 10
    # Construct the full URL for the search
    search_url = base_url + "search_query=all:e&start=0&max_results=" + str(nrPapers) + "&sortBy=lastUpdatedDate&sortOrder=descending"
    # get the raw data from the query
    data = urllib.request.urlopen(search_url)
    rawData = str(data.read().decode('utf-8'))

    # collect the links of newest papers
    papers = []
    entryMatches = re.finditer(r"<entry>(.*?)</entry>", rawData, flags=re.DOTALL)
    for entry in entryMatches:
        link = ""
        linkMatches = re.finditer(r"<id>(.*?)</id>", str(entry.group(1)), flags=re.DOTALL)
        for l in linkMatches:
            link = l.group(1)
        link = link.replace("\n", "")
        #print("Search complete!")
        papers.append(link)
    
    return papers


In [ ]:
papers = search()
for paper_link in papers:
    paper = next(arxiv.Search(id_list=[paper_link[21:]]).results())
    paper.download_pdf(dirpath="./fetched_pdfs")


In [ ]:
import fitz
fname = "./fetched_pdfs/2212.08015v1.First_detection_of_the_outer_edge_of_an_AGN_accretion_disc_Very_fast_multiband_optical_variability_of_NGC_4395_with_GTC_HiPERCAM_and_LT_IO_O.pdf"  # get document filename
doc = fitz.open(fname)  # open document
out = open(fname + ".txt", "wb")  # open text output
for page in doc:  # iterate the document pages
    text = page.get_text().encode("utf8")  # get plain text (is in UTF-8)
    out.write(text)  # write text of page
    out.write(bytes((12,)))  # write page delimiter (form feed 0x0C)
out.close()

In [ ]:
import os
import fitz
import json
from nucleus_api.rest import ApiException
dataset = "demos"
for fname in os.listdir("./fetched_pdfs"):
    path = "./fetched_pdfs/" + fname
    newdocument = {}
    newdocument["title"] = filename[13:-4].replace("_"," ")
    newdocument["time"] = "12/1/2023"
    newdocument["content"] = ""
    doc = fitz.open(path)
    for page in doc:  # iterate the document pages
        newdocument["content"] += str(page.getText().encode("utf8"))
    
    payload = nucleus_api.Appendjsonparams(dataset=dataset,
                                        document=newdocument)
    try:
        api_response = api_instance.post_append_json_to_dataset(payload)
        print(api_response.result)
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])  



In [ ]:
import pandas as pd

df = pd.read_csv('data.csv')
print(df)

In [ ]:
import json

path = "fetched_pdfs"
dataset = "demos"
nucleus.Dataset.delete_dataset(dataset)
for i in os.listdir(path):
    if i.endswith(".pdf"):
        api_response = api_instance.post_upload_file(path + "/" + i, dataset)
        fp = api_response.result
        print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)


In [ ]:
#compute topics:
payload = nucleus_api.Topics(dataset=dataset, num_topics=5)
api_response = api_instance.post_topic_api(payload) 
#create a json file topics.json containing only topic keywords, weights and strength
topics = [{'keywords':topic.keywords, 
           'keywords_weight':topic.keywords_weight, 
           'strength':topic.strength} for topic in api_response.result.topics]
print(json.dumps(topics, indent=4, ensure_ascii=False))
# with open('demo_topics.json', 'w') as outfile:
with open('/demo/demo_topics.json', 'w') as outfile:
    json.dump(topics, outfile, indent=4, ensure_ascii=False)

In [ ]:
import pandas as pd

df = pd.read_csv('data.csv')
print(df)